# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os

# Set your preferred name
USER = "Yuxuan"
# Set the data and output directories
DATA_ROOT = '/home/yxcheng'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Physical-Exercise-Response'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = False

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [2]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'CrawlData.ipynb',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Glioblastoma_(GBM)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Bladder_Cancer_(BLCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Cervical_Ca

If no match is found, jump directly to GEO in Part 2.2

In [5]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [6]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [7]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [8]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [9]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [10]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [11]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [12]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [13]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [14]:
gender_col = 'gender'

In [15]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [16]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [17]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.639586,-0.195527,-0.324695,0.193167,-0.572812
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.997486,-0.276027,-1.219395,1.850967,-0.471212
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.959514,0.714373,0.112305,-0.532833,-0.003812
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,1.013722,-1.587117,0.359267,2.59051,0.806623,0.812114,0.367373,-0.494595,-1.062233,0.108588
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,0.110214,1.469373,0.194605,1.404667,-0.290712


In [18]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [19]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lung-Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [20]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [21]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Lung-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [5]:
dataset = 'GEO'
trait_subdir = "Physical-Exercise-Response"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE181339',
 'GSE213029',
 'GSE84046',
 'GSE148152',
 'GSE156247',
 'GSE111553',
 'GSE101931',
 'GSE156249',
 'GSE116294',
 'GSE133910',
 'GSE106865',
 'GSE111554',
 'GSE142426',
 'GSE111552']

Repeat the below steps for all the accession numbers

In [59]:
cohort = accession_num = "GSE111552"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/home/yxcheng/GEO/Physical-Exercise-Response/GSE111552/GSE111552_family.soft.gz',
 '/home/yxcheng/GEO/Physical-Exercise-Response/GSE111552/GSE111552_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [60]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Blood and skeletal muscle gene signatures of exercise training in men (PBMC data set)"
!Series_summary	"Accounting for transcriptional features of endurance adaptations to training is important and could help elucidate the high variability in oxygen uptake (VO2) response. We aimed to identify whole-transcriptome signatures of an endurance training protocol in whole-blood (leukocytes), PBMCs and skeletal muscle tissue of the same group of individuals in a controlled environment."
!Series_overall_design	"Two groups of healthy untrained policemen recruits (G1, n=20, 26±4 yrs; G2, n= 21, 27±3 yrs) were selected. Blood (PAXgene and EDTA tubes) and muscle tissueforgene expression assays and peak VO2peak measures by cardiopulmonary exercise test were taken before and after 18 weeks of running (3 times/wk; 60min). RNA was extracted from whole-blood (leukocytes), PBMCs and muscle and was used for whole-genome expression analysis using AffymetrixGene Chip Human Transcriptome Array

In [61]:
clinical_data.head()

,!Sample_geo_accession,GSM3034063,GSM3034064,GSM3034065,GSM3034066,GSM3034067,GSM3034068,GSM3034069,GSM3034070,GSM3034071,...,GSM3034085,GSM3034086,GSM3034087,GSM3034088,GSM3034089,GSM3034090,GSM3034091,GSM3034092,GSM3034093,GSM3034094
0,!Sample_characteristics_ch1,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...,tissue: peripheral blood mononuclear cells (PB...
1,!Sample_characteristics_ch1,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,...,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male
2,!Sample_characteristics_ch1,age: 27,age: 19,age: 21,age: 30,age: 22,age: 21,age: 27,age: 29,age: 28,...,age: 27,age: 29,age: 28,age: 28,age: 28,age: 27,age: 30,age: 22,age: 29,age: 25
3,!Sample_characteristics_ch1,height: 1.81,height: 1.91,height: 1.77,height: 1.79,height: 1.78,height: 1.90,height: 1.81,height: 1.65,height: 1.77,...,height: 1.81,height: 1.65,height: 1.77,height: 1.64,height: 1.73,height: 1.68,height: 1.74,height: 1.84,height: 1.68,height: 1.67
4,!Sample_characteristics_ch1,body weight: 84.8,body weight: 100.6,body weight: 80.0,body weight: 71.7,body weight: 66.9,body weight: 70.3,body weight: 95.8,body weight: 72.6,body weight: 85.1,...,body weight: 91.9,body weight: 73.6,body weight: 87.8,body weight: 55.8,body weight: 89.6,body weight: 56.4,body weight: 75.6,body weight: 92.3,body weight: 64.9,body weight: 67.3


In [62]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['tissue: peripheral blood mononuclear cells (PBMCs)'],
 1: ['gender: male'],
 2: ['age: 27',
  'age: 19',
  'age: 21',
  'age: 30',
  'age: 22',
  'age: 29',
  'age: 28',
  'age: 25'],
 3: ['height: 1.81',
  'height: 1.91',
  'height: 1.77',
  'height: 1.79',
  'height: 1.78',
  'height: 1.90',
  'height: 1.65',
  'height: 1.64',
  'height: 1.73',
  'height: 1.68',
  'height: 1.74',
  'height: 1.84',
  'height: 1.67'],
 4: ['body weight: 84.8',
  'body weight: 100.6',
  'body weight: 80.0',
  'body weight: 71.7',
  'body weight: 66.9',
  'body weight: 70.3',
  'body weight: 95.8',
  'body weight: 72.6',
  'body weight: 85.1',
  'body weight: 56.9',
  'body weight: 89.0',
  'body weight: 55.3',
  'body weight: 76.8',
  'body weight: 95.5',
  'body weight: 67.4',
  'body weight: 65.4',
  'body weight: 80.6',
  'body weight: 93.6',
  'body weight: 83.0',
  'body weight: 69.7',
  'body weight: 66.3',
  'body weight: 71.6',
  'body weight: 91.9',
  'body weight: 73.6',
  'body weight: 

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [54]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Physical-Exercise-Response\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Physical-Exercise-Response\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [52]:
is_gene_availabe = True
trait_row_id = 3
age_row_id = 1
gender_row_id = 2

trait_type = 'binary'

In [53]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [56]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'comorbidity: osteoporosis':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip('y'))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'gender: female':
        return 0
    elif gender_string == 'gender: male':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23y'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(0, 23, None)

In [57]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:171: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:171: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)
/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:171: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  clinical_df = clinical_df.applymap(convert_fn)


,GSM7019507,GSM7019508,GSM7019509,GSM7019510,GSM7019511,GSM7019512,GSM7019513,GSM7019514,GSM7019515,GSM7019516,...,GSM7019528,GSM7019529,GSM7019530,GSM7019531,GSM7019532,GSM7019533,GSM7019534,GSM7019535,GSM7019536,GSM7019537
Osteoporosis,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
Age,63,64,63,48,70,62,58,57,60,57,...,70,76,62,73,71,59,62,47,76,54
Gender,0,1,0,0,1,1,1,1,0,0,...,0,0,0,1,0,0,1,0,0,0


### Genetic data preprocessing and final filtering

In [58]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,ID,GSM7019507,GSM7019508,GSM7019509,GSM7019510,GSM7019511,GSM7019512,GSM7019513,GSM7019514,GSM7019515,...,GSM7019528,GSM7019529,GSM7019530,GSM7019531,GSM7019532,GSM7019533,GSM7019534,GSM7019535,GSM7019536,GSM7019537
0,A_19_P00315452,8.401627,9.797257,8.487270,8.769897,8.097000,8.457107,8.842743,8.702670,7.915254,...,8.061999,8.117889,9.159816,9.487754,9.699332,8.200573,9.095698,9.422875,8.409377,9.587772
1,A_19_P00315492,6.398731,6.405888,5.985312,6.184363,6.170733,6.250089,6.303887,6.136184,6.086006,...,6.115380,6.250949,6.151105,5.935964,6.293813,6.430803,6.150817,6.331640,6.205212,6.811390
2,A_19_P00315493,7.806405,8.416414,8.369232,7.425126,9.318262,9.470389,9.316538,9.042451,8.309025,...,8.719534,8.723245,8.039626,8.512870,8.904771,7.988505,8.978647,8.276969,8.543147,10.224405
3,A_19_P00315502,6.485717,6.206045,6.378061,6.075918,6.057971,6.276804,5.878976,6.057259,6.102158,...,6.210990,6.106469,6.044730,5.859149,6.302471,6.127004,6.245157,6.196389,6.412637,6.325190
4,A_19_P00315506,7.505585,8.261315,7.723854,8.218476,8.149302,8.132288,7.953135,7.872324,8.138612,...,7.656146,8.073789,8.211345,8.475403,8.329081,7.773416,8.476529,8.361277,9.023644,7.931727


In [59]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['A_19_P00315452',
 'A_19_P00315492',
 'A_19_P00315493',
 'A_19_P00315502',
 'A_19_P00315506',
 'A_19_P00315518',
 'A_19_P00315519',
 'A_19_P00315529',
 'A_19_P00315541',
 'A_19_P00315543',
 'A_19_P00315551',
 'A_19_P00315581',
 'A_19_P00315584',
 'A_19_P00315593',
 'A_19_P00315603',
 'A_19_P00315625',
 'A_19_P00315627',
 'A_19_P00315631',
 'A_19_P00315641',
 'A_19_P00315647']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [60]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['A_19_P00315452', 'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315502', 'A_19_P00315506', 'A_19_P00315518', 'A_19_P00315519', 'A_19_P00315529', 'A_19_P00315541', 'A_19_P00315543', 'A_19_P00315551', 'A_19_P00315581', 'A_19_P00315584', 'A_19_P00315593', 'A_19_P00315603', 'A_19_P00315625', 'A_19_P00315627', 'A_19_P00315631', 'A_19_P00315641', 'A_19_P00315647']\n"

If not required, jump directly to the gene normalization step

In [61]:
requires_gene_mapping = True

In [62]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_001105533|gb|AK025639|gb|BC030538|tc|THC2601673', 'ens|ENST00000378463'], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped', 'chr7:120901888-120901947', 'chrX:39909128-39909069'], 'CYTOBAND': [nan, nan, nan, 'hs|7q31.31', 'hs|Xp11.4'], 'DESCRIPTION': [nan, nan, nan, 'Homo sapiens cadherin-like and PC-esterase domain containing 1 (CPED1), transcript variant 2, mRNA [NM_001105533]', 'BCL6 corepressor [So

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [63]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386', 'A_33_P3396872', 'A_33_P3267760'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_001105533', nan], 'GB_ACC': [nan, nan, nan, 'NM_001105533', nan], 'LOCUSLINK_ID': [nan, nan, nan, 79974.0, 54880.0], 'GENE_SYMBOL': [nan, nan, nan, 'CPED1', 'BCOR'], 'GENE_NAME': [nan, nan, nan, 'cadherin-like and PC-esterase domain containing 1', 'BCL6 corepressor'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.189652', nan], 'ENSEMBL_ID': [nan, nan, nan, nan, 'ENST00000378463'], 'ACCESSION_STRING': [nan, nan, n

In [64]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'GENE_SYMBOL'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [65]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [67]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [68]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 31 samples.


In [70]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Osteoporosis', the least common label is '1.0' with 4 occurrences. This represents 12.90% of the dataset.
The distribution of the feature 'Osteoporosis' in this dataset is fine.

Quartiles for 'Age':
  25%: 54.5
  50% (Median): 61.0
  75%: 64.5
Min: 47.0
Max: 84.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 9 occurrences. This represents 29.03% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [71]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [72]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [73]:
from utils import *

In [74]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE99624'

In [75]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE99624,True,True,False,True,True,48,
1,GSE224330,True,True,False,True,True,31,


In [95]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [96]:
merged_data.head()

,Osteoporosis,Age,Gender,A1BG,A1CF,A2BP1,A2LD1,A2M,A2ML1,A4GALT,...,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,psiTPTE22,tAKR
0,1.0,79.0,0.0,0.619183,0.801233,0.634907,0.837734,0.785291,0.825050,0.406635,...,0.446068,0.563331,0.311214,0.567529,0.506000,0.297853,0.492457,0.159661,0.227908,0.846619
1,1.0,77.0,0.0,0.658625,0.800475,0.649430,0.851926,0.789070,0.808220,0.398144,...,0.448991,0.602357,0.315149,0.557280,0.511341,0.313782,0.487427,0.162463,0.233809,0.845953
2,1.0,77.0,0.0,0.628286,0.820998,0.653810,0.860294,0.797432,0.801589,0.412759,...,0.459180,0.581639,0.327939,0.566793,0.528222,0.310803,0.496740,0.165988,0.221068,0.853355
3,1.0,68.0,0.0,0.653566,0.835262,0.664892,0.862146,0.800886,0.833859,0.408578,...,0.512569,0.603380,0.326088,0.534097,0.534841,0.338295,0.503251,0.168624,0.234407,0.869540
4,1.0,80.0,0.0,0.662453,0.818094,0.665005,0.865634,0.798020,0.848954,0.416313,...,0.502265,0.577554,0.361502,0.561915,0.521404,0.329367,0.483798,0.171065,0.250503,0.821742


In [97]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [98]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [99]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [100]:
if has_batch_effect:
    model_constructor = VariableSelection
    model_params = {}
else:
    model_constructor = Lasso
    model_params = {'alpha': 1.0, 'random_state': 42}

In [101]:

cv_mean, cv_std = cross_validation(X, Y, Z, model_constructor, model_params, target_type=trait_type)

The cross-validation accuracy is 62.22% ± 15.07%


In [102]:
# Train regression model on the whole dataset to identify significant genes
model = ResidualizationRegressor(model_constructor, model_params)
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)
model.fit(normalized_X, Y, normalized_Z)

### 5. Discussion and report

In [103]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

if has_batch_effect:
    report_result_from_lmm(model, feature_cols, TRAIT, condition, threshold=0.05, save_output=True,
                           output_dir=OUTPUT_DIR)
else:
    report_result_from_lasso(model, feature_cols, TRAIT, condition, save_output=True, output_dir=OUTPUT_DIR)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.1465
Found 0 significant genes associated with the trait 'Osteoporosis' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, corrected_p_value]
Index: []


/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
